# RSA经典题型

P.S.:
1. 默认读者已掌握RSA加密的基础知识，如果读者还不知道RSA算法，请先自行了解RSA算法的基本原理。
2. 请配置好密码学常用python环境，安装**pycryptodome**、**gmpy2**、**numpy**等常用数学库。
3. 常见的函数使用可以移步[这个网址](https://www.cnblogs.com/Lovechan/p/17622466.html)了解。

## 绪

RSA的题目以分解$p,q$为基本突破口。我们今后做的较多的题目也是分解$n$为核心。

本章主要列举一些RSA经典考点（不涉及高深数学知识和sagemath）。

## 已知$p,q,e,c$

[例题](./pqec.py)

最正常的RSA题目，按正常的解密算法来解就行。

In [ ]:
from Crypto.Util.number import *

p = ...
q = ...
e = ...
c = ...

phi = (p - 1)*(q - 1)
d = inverse(e, phi)
m = pow(c, d, p*q)

print(long_to_bytes(m))

## 已知$n(易分解),e,c$

[例题](./small_n.py)

$n$较小时可以通过[这个网站](http://www.factordb.com/)尝试分解。

![small_n_fact](./assets/small_n_factor.png)

这样就转化成了已知$p,q,e,c$的最基本的题型了。


## 已知$n,c,e(很小,通常为3)$

[例题](./low_e.py)

低加密指数攻击，可以暴力破解。

分析：
$$
c=m^e\pmod n\\
c=m^e+kn\\
m=(c+kn)^{1/e}
$$
我们从$0$开始枚举$k$，找到能被开方开尽的就是答案。这边使用`gmpy2`里的`iroot()`函数快速开多次幂。

In [ ]:
from Crypto.Util.number import *
from gmpy2 import iroot

e = ...
n = ...
c = ...

k = 0
while True:
    res = iroot(n*k + c, e)
    if res[1]:
        print(long_to_bytes(res[0]))
        break
    k += 1

## 已知$c,e,n$，和 $dp或dq$

[例题](./dp.py)

分析：
$$
dp=d+k_1\times (p-1)\\
dp\times e=e\times d+k_1(p-1)\\
$$
又$e\times d=k_2\times (p-1)\times (q-1)+1$，得：
$$
e\times d=dp\times e-k_1\times (p-1)=k_2\times (p-1)\times (q-1)+1\\
dp\times e=k_1\times (p-1)+k_2\times (p-1)\times (q-1)+1\\
\Rightarrow dp\times e=(p-1)\times [k_1+k_2\times (q-1)]+1\\
令K=k_1+k_2\times (q-1),得：\\
dp\times e=(p-1)\times K+1\\
\Rightarrow dp\times e-1=(p-1)\times K\\
$$
在这里界定一下$K$的范围：
$$
dp<p-1\\
\Rightarrow (p-1)\times K<e\times (p-1)-1
\Rightarrow K<e\\
$$
爆破就完事了。

In [ ]:
from Crypto.Util.number import *

n = ...
e = ...
c = ...
dp = ...

for k in range(1, e):
    if (dp*e - 1) % k == 0:
        p = ((dp*e - 1) // k) + 1
        if n % p == 0:
            q = n // p
            phi = (p - 1)*(q - 1)
            d = inverse(e, phi)
            print(long_to_bytes(pow(c, d, n)))